1. LIBRERÍAS

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.models import ConvNeXt_Base_Weights, convnext_base
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np
from tqdm import tqdm
import psutil 

2. CONFIGURACIÓN DEL DISPOSITIVO MPS

In [2]:
# Verificar si CUDA está disponible y seleccionar el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo:", device)

# Imprimir las características de la GPU si CUDA está disponible
if device.type == "cuda":
    print("Nombre de la GPU:", torch.cuda.get_device_name(0))
    print("Capacidad de computo:", torch.cuda.get_device_capability(0))
    print("Memoria total de la GPU: {:.2f} GB".format(torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
    print("Memoria libre: {:.2f} GB".format(torch.cuda.memory_reserved(0) / (1024 ** 3)))
    print("Memoria usada: {:.2f} GB".format(torch.cuda.memory_allocated(0) / (1024 ** 3)))
    print("Multiprocesadores:", torch.cuda.get_device_properties(0).multi_processor_count)
    print("Versión CUDA:", torch.version.cuda)
    print("Versión CuDNN:", torch.backends.cudnn.version())
else:
    print("CUDA no está disponible.")

Dispositivo: cuda
Nombre de la GPU: NVIDIA GeForce RTX 4090
Capacidad de computo: (8, 9)
Memoria total de la GPU: 23.55 GB
Memoria libre: 0.00 GB
Memoria usada: 0.00 GB
Multiprocesadores: 128
Versión CUDA: 12.4
Versión CuDNN: 90100


3. DIRECTORIOS Y TRANSFORMACIONES

In [3]:
data_dir = "/workspace/Training_dataset"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

4. CARGA DE DATOS

In [4]:
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform_train)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform_val_test)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform_val_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

class_names = train_dataset.classes

In [5]:
torch.cuda.empty_cache()
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_accumulated_memory_stats()

/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:391: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


5. CARGAR ConvNeXt

In [9]:
convnext = models.convnext_base(weights=models.ConvNeXt_Base_Weights.IMAGENET1K_V1).to(device)
convnext.features = convnext.features.to(device)
convnext = models.convnext_base(weights=models.ConvNeXt_Base_Weights.IMAGENET1K_V1).to(device)

# Ajustar la capa `classifier` según la salida real
convnext.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Asegurar tamaño fijo de salida
convnext.classifier = nn.Sequential(
    nn.Flatten(),                                   # Aplana la salida (batch_size, 1024, 1, 1) -> (batch_size, 1024)
    nn.Linear(1024, 512),                           # Ajustar a 512 neuronas intermedias
    nn.ReLU(),                                      # Activación intermedia
    nn.Dropout(p=0.7),                              # Dropout ajustado a 0.7
    nn.Linear(512, len(class_names))                # Capa final para clasificación
).to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(convnext.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
early_stopping_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)

6. ENTRENAMIENTO Y VALIDACIÓN

In [ ]:
train_loss, val_loss, train_acc, val_acc = [], [], [], []
start_time = time.time()

for epoch in range(100):
    print(f"\nEpoch {epoch + 1}/100")
    convnext.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc=f"Entrenando Epoca {epoch + 1}"):
        images = images.to(device)          
        labels = labels.to(device)          

        optimizer.zero_grad()
        outputs = convnext(images)
        loss = criterion(outputs, labels)   
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_train_loss = running_loss / len(train_loader)
    epoch_train_acc = correct / total
    train_loss.append(epoch_train_loss)
    train_acc.append(epoch_train_acc)

    # === Validación ===
    convnext.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)      
            labels = labels.to(device)     

            outputs = convnext(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_val_loss = running_loss / len(val_loader)
    epoch_val_acc = correct / total
    val_loss.append(epoch_val_loss)
    val_acc.append(epoch_val_acc)

    # Actualizar el scheduler
    scheduler.step()
    early_stopping_scheduler.step(epoch_val_loss)

    print(f"Epoca {epoch + 1} -> "
          f"Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}, "
          f"Train Acc: {epoch_train_acc:.4f}, Val Acc: {epoch_val_acc:.4f}")

end_time = time.time()
training_time = (end_time - start_time) / 60
print(f"Tiempo total de entrenamiento: {training_time:.2f} minutos")


Epoch 1/100


Entrenando Epoca 1: 100%|██████████| 657/657 [01:54<00:00,  5.73it/s]


Epoca 1 -> Train Loss: 0.4369, Val Loss: 0.2537, Train Acc: 0.8691, Val Acc: 0.9231

Epoch 2/100


Entrenando Epoca 2: 100%|██████████| 657/657 [01:54<00:00,  5.73it/s]


Epoca 2 -> Train Loss: 0.2273, Val Loss: 0.1464, Train Acc: 0.9362, Val Acc: 0.9556

Epoch 3/100


...

Epoca 98 -> Train Loss: 0.0068, Val Loss: 0.0605, Train Acc: 0.9984, Val Acc: 0.9898

Epoch 99/100


Entrenando Epoca 99: 100%|██████████| 657/657 [01:53<00:00,  5.80it/s]


Epoca 99 -> Train Loss: 0.0054, Val Loss: 0.0598, Train Acc: 0.9983, Val Acc: 0.9900

Epoch 100/100


Entrenando Epoca 100: 100%|██████████| 657/657 [01:53<00:00,  5.80it/s]


Epoca 100 -> Train Loss: 0.0047, Val Loss: 0.0603, Train Acc: 0.9987, Val Acc: 0.9902
Tiempo total de entrenamiento: 199.72 minutos


7. GUARDAR EL MODELO

In [ ]:

torch.save(convnext.state_dict(), "/workspace/Models/ConvNeXt_model.pth")

8. EVALUACIÓN EN EL CONJUNTO DE PRUEBA

In [ ]:
os.makedirs("ConvNeXt", exist_ok=True)

convnext.eval()
y_true, y_pred = [], []
embeddings = []

with torch.no_grad():
    for images, labels in test_loader:
        # Mover datos a la GPU si está disponible
        images, labels = images.to(device), labels.to(device)
        
        # Extraer embeddings y mover a la CPU
        features = convnext.features(images)  # Cambiado a convnext
        embeddings.append(features.view(features.size(0), -1).detach().cpu().numpy())
        
        # Generar predicciones
        outputs = convnext(images)  # Cambiado a convnext
        _, preds = torch.max(outputs, 1)
        
        # Mover predicciones y etiquetas a la CPU
        y_true.extend(labels.detach().cpu().numpy())
        y_pred.extend(preds.detach().cpu().numpy())

# Concatenar embeddings y convertirlos a un array
embeddings = np.concatenate(embeddings)

# Generar reporte de clasificación
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
report_df = pd.DataFrame(report).transpose()

# Guardar reporte en un archivo Excel
report_df.to_excel("/workspace/ConvNext/Metricas_ConvNeXt.xlsx")

print("Reporte guardado en: /workspace/ConvNext/Metricas_ConvNeXt.xlsx")


9. MATRIZ DE CONFUSIÓN

In [ ]:
# Calcular la matriz de confusión y normalizarla
conf_matrix = confusion_matrix(y_true, y_pred)
conf_matrix_norm = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

# Crear el gráfico de la matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(
    conf_matrix_norm,
    annot=True,
    fmt='.2%',
    cmap='PuBuGn',  
    xticklabels=class_names,
    yticklabels=class_names,
    linewidths=0.5,
    linecolor='black',
    cbar_kws={'aspect': 30, 'shrink': 0.8} 
)

# Títulos y etiquetas
plt.title('Matriz de Confusión Normalizada (%) - ConvNeXt', fontsize=16, fontweight='bold', family='serif')
plt.xlabel('Predicción', fontsize=14, family='serif')
plt.ylabel('Valor Real', fontsize=14, family='serif')
plt.xticks(rotation=45, fontsize=12, family='serif')
plt.yticks(rotation=0, fontsize=12, family='serif')

# Ajustes adicionales
plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
plt.tight_layout()

# Guardar el gráfico
plt.savefig("/workspace/ConvNeXt/Matriz_confusion.svg", format='svg', dpi=300, bbox_inches='tight')
plt.savefig("/workspace/ConvNeXt/Matriz_confusion.png", dpi=300, bbox_inches='tight')
plt.show()

print("Matriz de confusión guardada en 'ConvNeXt'.")


10. GRAFICAR MÉTRICAS

In [ ]:
# Suavizado de curvas
def suavizar(valores, peso=0.8):
    smoothed = []
    ultimo = valores[0]
    for punto in valores:
        smoothed_val = ultimo * peso + (1 - peso) * punto
        smoothed.append(smoothed_val)
        ultimo = smoothed_val
    return smoothed

train_acc_smooth = suavizar(train_acc)
val_acc_smooth = suavizar(val_acc)
train_loss_smooth = suavizar(train_loss)
val_loss_smooth = suavizar(val_loss)

# Gráfica 1: Accuracy durante el entrenamiento
plt.figure(figsize=(10, 8))
plt.plot(train_acc_smooth, label='Accuracy de entrenamiento', linestyle='-', linewidth=2, color='mediumseagreen')
plt.plot(val_acc_smooth, label='Accuracy de validación', linestyle='-', linewidth=2, color='coral')
plt.plot(train_acc, 'o--', alpha=0.7, color='lightgreen', label='Valores reales de entrenamiento')
plt.plot(val_acc, 'o--', alpha=0.7, color='lightcoral', label='Valores reales de validación')
plt.xlabel('Épocas', fontsize=14, family='serif')
plt.ylabel('Accuracy', fontsize=14, family='serif')
plt.title('Accuracy durante el entrenamiento (ConvNeXt)', fontsize=16, fontweight='bold', family='serif')
plt.ylim([0.9, 1.0])
plt.legend(loc='lower right', fontsize=12, prop={'family': 'serif'})
plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
plt.tight_layout()
plt.savefig("/workspace/ConvNeXt/Accuracy_ConvNeXt.svg", format='svg', dpi=300, bbox_inches='tight')
plt.savefig("/workspace/ConvNeXt/Accuracy_ConvNeXt.png", dpi=300, bbox_inches='tight')
plt.show()

# Gráfica 2: Pérdida durante el entrenamiento
plt.figure(figsize=(10, 8))
plt.plot(train_loss_smooth, label='Pérdida de entrenamiento', linestyle='-', linewidth=2, color='mediumseagreen')
plt.plot(val_loss_smooth, label='Pérdida de validación', linestyle='-', linewidth=2, color='coral')
plt.plot(train_loss, 'o--', alpha=0.7, color='lightblue', label='Valores reales de entrenamiento')
plt.plot(val_loss, 'o--', alpha=0.7, color='lightcoral', label='Valores reales de validación')
plt.xlabel('Épocas', fontsize=14, family='serif')
plt.ylabel('Pérdida', fontsize=14, family='serif')
plt.title('Pérdida durante el entrenamiento (ConvNeXt)', fontsize=16, fontweight='bold', family='serif')
plt.ylim([0, 0.35])
plt.legend(loc='lower left', fontsize=12, prop={'family': 'serif'})
plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
plt.tight_layout()
plt.savefig("/workspace/ConvNeXt/Perdida_ConvNeXt.svg", format='svg', dpi=300, bbox_inches='tight')
plt.savefig("/workspace/ConvNeXt/Perdida_ConvNeXt.png", dpi=300, bbox_inches='tight')
plt.show()

# Gráfica 3: Comparación de Pérdida y Accuracy durante el entrenamiento
plt.figure(figsize=(10, 8))
plt.plot(train_loss_smooth, label='Pérdida de entrenamiento', linestyle='-', linewidth=2, color='mediumseagreen')
plt.plot(val_loss_smooth, label='Pérdida de validación', linestyle='-', linewidth=2, color='coral')
plt.plot(train_acc_smooth, label='Accuracy de entrenamiento', linestyle='--', linewidth=2, color='lightgreen')
plt.plot(val_acc_smooth, label='Accuracy de validación', linestyle='--', linewidth=2, color='lightcoral')
plt.plot(train_loss, 'o--', alpha=0.7, color='lightblue', label='Valores reales de pérdida (entrenamiento)')
plt.plot(val_loss, 'o--', alpha=0.7, color='lightcoral', label='Valores reales de pérdida (validación)')
plt.plot(train_acc, 'o--', alpha=0.7, color='lightgreen', label='Valores reales de Accuracy (entrenamiento)')
plt.plot(val_acc, 'o--', alpha=0.7, color='lightcoral', label='Valores reales de Accuracy (validación)')
plt.xlabel('Épocas', fontsize=14, family='serif')
plt.ylabel('Valores', fontsize=14, family='serif')
plt.title('Comparación de Pérdida y Accuracy durante el entrenamiento (ConvNeXt)', fontsize=16, fontweight='bold', family='serif')
plt.legend(loc='best', fontsize=12, prop={'family': 'serif'})
plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
plt.tight_layout()
plt.savefig("/workspace/ConvNeXt/Comparacion_Perdida_Accuracy_ConvNeXt.svg", format='svg', dpi=300, bbox_inches='tight')
plt.savefig("/workspace/ConvNeXt/Comparacion_Perdida_Accuracy_ConvNeXt.png", dpi=300, bbox_inches='tight')
plt.show()

11. ANÁLISIS DE VECTORES (Vector Analysis)

In [ ]:
print("Generando análisis de vectores...")

convnext.eval()
y_true, y_pred, embeddings = [], [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = convnext(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

        # Extraer y almacenar embeddings
        features = convnext.forward_features(images)  
        embeddings.append(features.view(features.size(0), -1).cpu().numpy())

# Unificar embeddings en una matriz NumPy
embeddings = np.concatenate(embeddings)

# Calcular F1 Scores por imagen
f1_scores = [f1_score([true], [pred], average='weighted') for true, pred in zip(y_true, y_pred)]
print("Tamaño de f1_scores:", len(f1_scores))

# Verificar y ajustar la forma de embeddings
print("Forma de embeddings antes de reshape:", embeddings.shape)
if len(embeddings.shape) == 1:
    embeddings = embeddings.reshape(-1, 1)
print("Forma de embeddings después de reshape:", embeddings.shape)

# Aplicar t-SNE con parámetros ajustados
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=3000)
embeddings_2d = tsne.fit_transform(embeddings)

print("Tamaño de embeddings_2d:", embeddings_2d.shape)
print("Tamaño de f1_scores:", len(f1_scores))

# Generar el gráfico t-SNE con la paleta 'viridis'
plt.figure(figsize=(12, 10))
scatter = plt.scatter(
    x=embeddings_2d[:, 0], 
    y=embeddings_2d[:, 1], 
    c=f1_scores, 
    cmap='viridis', 
    s=20,           
    alpha=0.8,       
    edgecolor='k'   
)
cbar = plt.colorbar(scatter, label='F1 Score', aspect=30)
cbar.ax.tick_params(labelsize=12)

# Personalización avanzada del gráfico
plt.title('Análisis t-SNE basado en F1 Scores por imagen - ConvNeXt', fontsize=16, fontweight='bold', family='serif')
plt.xlabel('Componente t-SNE 1', fontsize=14, family='serif')
plt.ylabel('Componente t-SNE 2', fontsize=14, family='serif')
plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
plt.xticks(fontsize=12, family='serif')
plt.yticks(fontsize=12, family='serif')

# Guardar el gráfico en formato SVG para alta calidad en presentaciones
plt.savefig("/workspace/ConvNeXt/Vector_Analysis_ConvNeXt_Viridis.svg", format='svg', dpi=300, bbox_inches='tight')
plt.savefig("/workspace/ConvNeXt/Vector_Analysis_ConvNeXt_Viridis.png", dpi=300, bbox_inches='tight')
plt.show()

# Guardar los datos procesados
df_embeddings = pd.DataFrame(embeddings_2d, columns=['Componente_1', 'Componente_2'])
df_embeddings['F1_Score'] = f1_scores
df_embeddings.to_csv("/workspace/ConvNeXt/Embeddings_y_F1Scores_Viridis.csv", index=False)

print("Análisis de vectores completado. Resultados guardados en '/workspace/ConvNeXt'.")


12. EVALUACIÓN EN EL CONJUNTO DE PRUEBA

In [ ]:
print("Generando predicciones y calculando scores...")
convnext.eval()
y_true, y_pred, y_scores = [], [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = convnext(images)
        y_scores.extend(outputs.cpu().numpy())
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Preparación de Datos para la Curva ROC
y_scores = np.array(y_scores)
y_true_bin = label_binarize(y_true, classes=list(range(len(class_names))))

fpr, tpr, roc_auc = {}, {}, {}
for i in range(len(class_names)):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_scores[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Cálculo de Promedio Macro y Micro AUC
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(len(class_names))]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(len(class_names)):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= len(class_names)
macro_auc = auc(all_fpr, mean_tpr)

fpr_micro, tpr_micro, _ = roc_curve(y_true_bin.ravel(), y_scores.ravel())
micro_auc = auc(fpr_micro, tpr_micro)

# Generación del Gráfico de Curvas ROC
plt.figure(figsize=(12, 10))
colors = plt.cm.tab10(np.linspace(0, 1, len(class_names)))

for i, color in zip(range(len(class_names)), colors):
    plt.plot(fpr[i], tpr[i], color=color, linewidth=2,
             label=f'Clase {class_names[i]} (AUC = {roc_auc[i]:.2f})')

# Agregar curvas promedio Macro y Micro
plt.plot(all_fpr, mean_tpr, linestyle='--', color='black', linewidth=2,
         label=f'Promedio Macro AUC = {macro_auc:.2f}')
plt.plot(fpr_micro, tpr_micro, linestyle=':', color='red', linewidth=2,
         label=f'Promedio Micro AUC = {micro_auc:.2f}')

# Configuración Avanzada del Gráfico
plt.plot([0, 1], [0, 1], 'k--', linewidth=1.5)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de falsos positivos', fontsize=12)
plt.ylabel('Tasa de verdaderos positivos', fontsize=12)
plt.title('Curvas ROC para cada clase con promedio Macro y Micro - ConvNeXt', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)

# Guardar el gráfico con alta resolución
plt.tight_layout()
plt.savefig("/workspace/ConvNeXt/Curva_ROC_AUC_ConvNeXt.png", dpi=300, bbox_inches='tight')
plt.show()

# Crear listas separadas para clases, AUC y tipo
clases = class_names + ['Promedio Macro', 'Promedio Micro']
auc_scores = [roc_auc[i] for i in range(len(class_names))] + [macro_auc, micro_auc]
tipo = ['Individual'] * len(class_names) + ['Promedio Macro', 'Promedio Micro']

# Asegurar que todas las listas tengan la misma longitud
assert len(clases) == len(auc_scores) == len(tipo), "Las listas no tienen la misma longitud"

# Crear el DataFrame corregido
roc_data = {
    'Clase': clases,
    'AUC': auc_scores,
    'Tipo': tipo
}
df_roc = pd.DataFrame(roc_data)
df_roc.to_csv("/workspace/ConvNeXt/Curvas_ROC_AUC_Resumen_ConvNeXt.csv", index=False)

print("Curvas ROC generadas y guardadas en '/workspace/ConvNeXt'.")

13. GUARDAR INFORME

In [ ]:
with open("/workspace/ConvNeXt/Detalles_entrenamiento_ConvNeXt.txt", "w") as f:
    f.write(f"Tiempo total de entrenamiento: {training_time:.2f} segundos\n")
    f.write(f"Dispositivo: {device}\n")
    f.write("Modelo: ConvNeXt\n")
    f.write("Directorio del modelo: /workspace/Models/ConvNeXt_model.pth\n")
    f.write(f"Parámetros: {sum(p.numel() for p in convnext.parameters())}\n")
    f.write("Métricas:\n")
    f.write(report_df.to_string())
    f.write("\n\nMatriz de Confusión guardada en: ConvNeXt/Matriz_confusion_ConvNeXt.png\n")
    f.write("Análisis de Vectores guardado en: /workspace/ConvNeXt/Vector_Analysis_ConvNeXt.png\n")
    f.write("Curvas ROC guardadas en: /workspace/ConvNeXt/Curva_ROC_AUC_ConvNeXt.png\n")
    f.write("Resumen AUC guardado en: /workspace/ConvNeXt/Curvas_ROC_AUC_Resumen_ConvNeXt.csv\n")

print("Entrenamiento y evaluación completados. Resultados guardados en 'ConvNeXt'.")